In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_data= pd.read_csv("/kaggle/input/covid-train-dataset/LitCovid.train.csv")

In [ ]:
df_data.head()

In [ ]:
#select  data without unnamed colum
df_data.loc[:, ~df_data.columns.str.contains('^Unnamed')]

In [ ]:
#drop the unnamed colum
df_data.drop(df_data.columns[df_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df_data.head()

In [ ]:
# drop rows with at least one Nan value (Null value)
df_data.dropna() 

In [ ]:
#remove missing values in text colum
#add a column encoding the human_label as category_id
#also create a couple of dictionaries

from io import StringIO
col = ['human_label', 'text']
df_data = df_data[col]
df_data = df_data[pd.notnull(df_data['text'])]
df_data.columns = ['human_label', 'text']
df_data ['category_id'] = df_data ['human_label'].factorize()[0]
category_id_df = df_data[['human_label', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'human_label']].values)
df_data .head(10)

In [ ]:
#text preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df_data.text).toarray()
labels = df_data.category_id
features.shape

In [ ]:
df_data .head(20)

In [ ]:
#find the terms that are the most correlated with each of the human_label:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for human_label, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(human_label))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))


In [ ]:
# we first transformed the “text” into a vector of numbers.We explored vector representations such as TF-IDF weighted vectors.
#After having this vector representations of the text we can train supervised classifiers to train unseen “text” and predict the “human_label” on which they fall.
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df_data['text'], df_data['human_label'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
#After fitting the training set, let’s make some predictions.
df_data[df_data['text'] == "Blockchain and Artificial Intelligence Technology for Novel Coronavirus Disease-19 Self-Testing. The novel coronavirus disease 19 (COVID-19) is rapidly spreading with a rising death toll and transmission rate reported in high income countries rather than in low income countries. The overburdened healthcare systems and poor disease surveillance systems in resource-limited settings may struggle to cope with this COVID-19 outbreak and this calls for a tailored strategic response for these settings. Here, we recommend a low cost blockchain and artificial intelligence-coupled self-testing and tracking systems for COVID-19 and other emerging infectious diseases. Prompt deployment and appropriate implementation of the proposed system have the potential to curb the transmissions of COVID-19 and the related mortalities, particularly in settings with poor access to laboratory infrastructure."]

In [ ]:
#For Decession tree Model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
dt= DecisionTreeClassifier(min_samples_split = 100, criterion='entropy')
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df_data.index, test_size=0.33, random_state=0)
dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
y_pred
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Desicion Tree: ", round(score, 1), "%" )

#confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.human_label.values, yticklabels=category_id_df.human_label.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#for precession,recall,F1 score ...
print(classification_report(y_test, y_pred))

In [ ]:
#For SVM  Model one versa Rest

from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
model = LinearSVC()
ovr=OneVsRestClassifier(model)

X_train, y_train= make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=2, random_state=1)
ovr.fit(X_train, y_train)
#y_pred = ovr.predict(y_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using SVM Classifier: ", round(score, 1), "%" )

#confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.human_label.values, yticklabels=category_id_df.human_label.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#for precession,recall,F1 score ...
print(classification_report(y_test, y_pred))

In [ ]:
y_pred = ovr.predict('Blockchain')